# HW 1 - Data Platform IDC

## Step 1: Setup Cassandra Cluster locally -

> Code is in markdown as we don't want to run this from the notebook
> The setup is oriented for mac env

Make sure Docker Desktop is installed. If not, download and install Docker from the official website [Get Docker | Docker Docs](https://docs.docker.com/get-docker/).

Download docker image for cassandra
```bash
docker pull cassandra:latest
```

Run container locally open for conncetion from the notebook
```bash
docker run --volume=/var/lib/cassandra --restart=no -p 127.0.0.1:9042:9042 -p 127.0.0.1:9160:9160 --name hw-cass -d cassandra:latest
```

Connect to run `cqlsh` locally
```bash
docker exec -it hw-cass cqlsh
```

## Step 2: setup python environment

install python
```bash
brew install python
```

install required libraries

cassandra driver
```bash
pip3 install cassandra-driver
```

## Step 3: Chossing the dataset

### Data set content - [dataset link](https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset/data)
The Book-Crossing dataset comprises 3 files.


**Users**
- Contains the users. Note that user IDs (User-ID) have been anonymized and map to integers. Demographic data is provided (Location, Age) if available. Otherwise, these fields contain NULL-values.

**Books**
- Books are identified by their respective ISBN. Invalid ISBNs have already been removed from the dataset. 
- Moreover, some content-based information is given (Book-Title, Book-Author, Year-Of-Publication, Publisher), obtained from Amazon Web Services. 
- Note that in case of several authors, only the first is provided. URLs linking to cover images are also given, appearing in three different versions (Image-URL-S, Image-URL-M, Image-URL-L), i.e., small, medium, large. These URLs point to the Amazon web site.

**Ratings**
- Contains the book rating information. Ratings (Book-Rating) are either explicit, expressed on a scale from 1-10 (higher values denoting higher appreciation), or implicit, expressed by 0.

#### In this project
The data set is store in the `Data` folder - where the names of the files are corresponding to the explnation above 

#### Dataset selection reason
TODO: Describe why data set was selected.

TODO: Describe size of data set

## Step 4: Cassendra Database design
TODO: describe the partition keys, clustering columns etc...


## Step 5: Setup keyspace and tables

First - We would like to load all the libraries need for ingestion and working with the cassandra DB

In [1]:
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement, BatchStatement

import csv
import concurrent.futures

Connect to our cassandra instance.

In [2]:
cluster = Cluster(contact_points=['127.0.0.1'])
session = cluster.connect()

Create out key space `books`.
We will use `SimpleStrategy` and `replication_factor` = `1` as they serve us well for the purpose of this excsrsize - as we are not looking for any HA or significant scale

In [3]:
session.execute("CREATE KEYSPACE IF NOT EXISTS books WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 };")
session.execute("USE books");

Make sure keyspace created

In [15]:
session.execute("DESCRIBE books;").one()

Row(keyspace_name='books', type='keyspace', name='books', create_statement="CREATE KEYSPACE books WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '1'}  AND durable_writes = true;")

Use the keyspace

In [14]:
session.execute("USE books");

In [ ]:
session.execute("""
    CREATE TABLE IF NOT EXISTS books (
        ISBN text PRIMARY KEY,
        Book_Title text,
        Book_Author text,
        Year_Of_Publication int,
        Publisher text,
        Image_URL_S text,
        Image_URL_M text,
        Image_URL_L text
    )
""")

In [ ]:
session.execute("""
    CREATE TABLE IF NOT EXISTS book_ratings (
        User_ID text,
        ISBN text,
        Book_Rating int,
        PRIMARY KEY(User_ID, ISBN)
    )
""")

In [ ]:
session.execute("""
    CREATE TABLE IF NOT EXISTS users (
        User_ID text PRIMARY KEY,
        Location text,
        Age float
    )
""")

## Step 6: Data ingestion

*Data ingestion function*
`load_data` function reads data from a CSV file, splits it into batches, and inserts the data into a table using concurrency.

Batches and concurrency are important for improving the performance and efficiency of inserting large amounts of data into a database table.

When inserting data into a database table, it is possible to insert row by row, or to insert multiple rows at once as a batch. Inserting data in a batch greatly improves the performance of inserting data into the database, as it reduces the number of round trips to the database and can ensure that the data is consistent. This is more efficient than inserting one row at a time, which can be slow and can lead to unnecessary overhead.

Concurrency is important because it allows multiple threads to be used when inserting data into a database table. This improves performance by allowing multiple inserts to happen simultaneously, which can greatly increase the speed of inserting large amounts of data. Without concurrency, each insert operation would have to wait for the previous insert to complete, leading to a slower overall process.

In summary, using batches and concurrency can greatly improve the performance and efficiency of inserting large amounts of data into a database table, resulting in faster insert times and better use of system resources.



In [3]:
def load_data(csv_file, insert_query, batch_size=100, concurrency=20):        
    
    def insert_data(rows):
        batch = BatchStatement()
        for row in rows:
            age = float(row[2]) if row[2] else None
            batch.add(SimpleStatement(insert_query), (row[0], row[1], age))
        session.execute(batch)
    
    with open(csv_file, "r") as f:
        next(f)  # Skip the header row.
        reader = list(csv.reader(f))

    # split to batches
    rows = [reader[i: i + batch_size] for i in range(0, len(reader), batch_size)] 
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=concurrency) as executor:
        executor.map(insert_data, rows)

In [ ]:
ratings_query = """
            INSERT INTO book_ratings (User_ID, ISBN, Book_Rating)
            VALUES (%s, %s, %s)
            """
load_data('Data/Ratings.csv', ratings_query)

In [ ]:
users_query = """
                INSERT INTO users (User_ID, Location, Age)
                VALUES (%s, %s, %s)
                """
load_data('Data/Users.csv', users_query)

In [ ]:
books_query = """
            INSERT INTO books (ISBN, Book_Title, Book_Author, Year_Of_Publication, Publisher, Image_URL_S, Image_URL_M, Image_URL_L)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """,
load_data('Data/Books.csv', books_query)

In [22]:
import csv

with open('Data/Books.csv', 'r') as f:
    next(f)  # Skip the header row.
    reader = csv.reader(f)
    for row in reader:
        session.execute(
            """
            INSERT INTO books (ISBN, Book_Title, Book_Author, Year_Of_Publication, Publisher, Image_URL_S, Image_URL_M, Image_URL_L)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """,
            (row[0], row[1], row[2], int(row[3]), row[4], row[5], row[6], row[7])
        )


KeyboardInterrupt: 

In [ ]:


with open('Data/Ratings.csv', 'r') as f:
    next(f)  # Skip the header row.
    reader = csv.reader(f)
    for row in reader:
        session.execute(
            """
            INSERT INTO book_ratings (User_ID, ISBN, Book_Rating)
            VALUES (%s, %s, %s)
            """,
            (row[0], row[1], int(row[2]))
        )

In [36]:
ratings_query =             """
            INSERT INTO book_ratings (User_ID, ISBN, Book_Rating)
            VALUES (%s, %s, %s)
            """
load_data('Data/Ratings.csv', ratings_query)

In [35]:


def load_data(csv_file, insert_query):
        
    def insert_data(rows):
        batch = BatchStatement()
        for row in rows:
            age = float(row[2]) if row[2] else None
            batch.add(SimpleStatement(insert_query), (row[0], row[1], age))
        session.execute(batch)
        print("exec")
    
    with open(csv_file, "r") as f:
        next(f)  # Skip the header row.
        reader = list(csv.reader(f))
    
    # Adjust the number of rows for each batch and number of workers as needed
    num_rows_per_batch = 100
    rows = [reader[i: i + num_rows_per_batch] for i in range(0, len(reader), num_rows_per_batch)] 
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
        executor.map(insert_data, rows)



In [ ]:
users_query = """
                INSERT INTO user_data (User_ID, Location, Age)
                VALUES (%s, %s, %s)
                """
load_data('Data/Users.csv', users_query)

In [37]:
books_query = """
            INSERT INTO books (ISBN, Book_Title, Book_Author, Year_Of_Publication, Publisher, Image_URL_S, Image_URL_M, Image_URL_L)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """,
load_data('Data/Books.csv', books_query)

In [33]:
count_query = "SELECT COUNT(*) FROM user_data"
result = session.execute(count_query)
result.one()

Row(count=278858)

In [38]:
count_query = "SELECT COUNT(*) FROM books"
result = session.execute(count_query)
result.one()

Row(count=157963)

## Step 3: Creating Tables

In [ ]:
USE university;

In [ ]:
CREATE TABLE students (
  student_id UUID PRIMARY KEY,
  name TEXT,
  email TEXT,
  enrollment_year INT
);

In [ ]:
CREATE TABLE courses (
  course_id UUID PRIMARY KEY,
  course_name TEXT,
  lecturer_id UUID,
  credits INT
);

In [ ]:
CREATE TABLE lecturers (
  lecturer_id UUID PRIMARY KEY,
  name TEXT,
  department TEXT
);

In [ ]:
CREATE TABLE course_enrollments (
  course_id UUID,
  student_id UUID,
  enrollment_date DATE,
  PRIMARY KEY (course_id, student_id)
);

In [ ]:
# To validate you created the tables
cqlsh:university> DESCRIBE tables;

In [ ]:
# The output should be:
course_enrollments  courses  lecturers  students

## Step 4: Inserting data

In [ ]:
# Inserting data into students
INSERT INTO students (student_id, name, email, enrollment_year) VALUES (550e8400-e29b-41d4-a716-446655440000, 'John Doe', 'john.doe@email.com', 2021);
INSERT INTO students (student_id, name, email, enrollment_year) VALUES (652e8500-f39c-42d5-b517-557655450001, 'Jane Smith', 'jane.smith@email.com', 2020);
INSERT INTO students (student_id, name, enrollment_year) VALUES (753e8600-e49d-53e6-c628-668655460002, 'Alice Johnson', 2019);

In [ ]:
# Inserting data into lecturers
INSERT INTO lecturers (lecturer_id, name, department) VALUES (860e9700-a59e-63f7-d739-779655470003, 'Dr. James Watson', 'Biology');
INSERT INTO lecturers (lecturer_id, name, department) VALUES (961e9800-b69f-73c8-e840-889655480004, 'Dr. Emma Wilson', 'Physics');
INSERT INTO lecturers (lecturer_id, name) VALUES (072e9900-d710-83e9-f951-990655490005, 'Prof. Michael Brown');

In [ ]:
# Inserting data into courses
INSERT INTO courses (course_id, lecturer_id, course_name, credits) VALUES (183fa000-f821-93a0-b062-a96655650006, 860e9700-a59e-63f7-d739-779655470003, 'Biology 101', 3);
INSERT INTO courses (course_id, lecturer_id, course_name, credits) VALUES (284fb000-a932-a4b1-c173-b97755660007, 961e9800-b69f-73c8-e840-889655480004, 'Physics 101', 4);
INSERT INTO courses (course_id, lecturer_id, course_name, credits) VALUES (385fc000-d043-b5e2-f284-c98855670008, 072e9900-d710-83e9-f951-990655490005, 'Mathematics 101', 3);

In [ ]:
# Inserting data into course_enrollments
INSERT INTO course_enrollments (course_id, student_id, enrollment_date) VALUES (183fa000-f821-93a0-b062-a96655650006, 550e8400-e29b-41d4-a716-446655440000, '2023-10-01');
INSERT INTO course_enrollments (course_id, student_id, enrollment_date) VALUES (284fb000-a932-a4b1-c173-b97755660007, 652e8500-f39c-42d5-b517-557655450001, '2023-09-15');
INSERT INTO course_enrollments (course_id, student_id) VALUES (385fc000-d043-b5e2-f284-c98855670008, 753e8600-e49d-53e6-c628-668655460002);

## Step 5: Querying the data

In [1]:
# Get all students
SELECT * FROM STUDENTS

>>
 student_id                           | email                | enrollment_year | name
--------------------------------------+----------------------+-----------------+---------------
 753e8600-e49d-53e6-c628-668655460002 |                 null |            2019 | Alice Johnson
 652e8500-f39c-42d5-b517-557655450001 | jane.smith@email.com |            2020 |    Jane Smith
 550e8400-e29b-41d4-a716-446655440000 |   john.doe@email.com |            2021 |      John Doe
 (3 rows)

In [ ]:
# Get the list of students who enrolled after 2020 along with their email and the courses they enrolled in
SELECT student_id, name, email
FROM students
WHERE enrollment_year > 2020 ALLOW FILTERING;

>>
 student_id                           | name     | email
--------------------------------------+----------+--------------------
 550e8400-e29b-41d4-a716-446655440000 | John Doe | john.doe@email.com

(1 rows)

In [ ]:
# Find all courses and count of students enrolled in them
SELECT course_id, COUNT(student_id) as student_count
FROM course_enrollments
GROUP BY course_id;

>>
 course_id                            | student_count
--------------------------------------+---------------
 183fa000-f821-93a0-b062-a96655650006 |             1
 385fc000-d043-b5e2-f284-c98855670008 |             1
 284fb000-a932-a4b1-c173-b97755660007 |             1

(3 rows)